# Experimental Code

<i>Version 1.0</i>

## Dependencies

Algebra's are defined in JSON format.

In [8]:
import json

The basic elements of an <b>Algebra</b>, as defined here, are subsets of a finite set of relations (<b>relsets</b>). Relsets are represented here using [the <b>bitset</b> package](https://bitsets.readthedocs.io/en/stable/).

In [9]:
from bitsets import bitset

Relsets are used to represent spatio-temporal constraints between spatio-temporal entities.  This can be represented as a graph, where the nodes are the entities and the edges are "labeled" with the constraints.  The Graph class in [the NetworkX package](https://networkx.github.io/) is extended to represent a Constraint Graph.

In [10]:
import networkx as nx

We use os.path, along with an environment variable, PYPROJ, to provide an OS/env-agnostic way of finding the qualreas code.  You must define or set PYPROJ appropriately for your own environment.

In [11]:
import os

In [12]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

## Algebras use Bitsets to Represent Sets of Relations

In [13]:
# TODO: Don't embed the abbreviation dictionary in code; create a file for it
def abbrev(term_list):
    abbrev_dict = {"Point":"Pt",
                   "ProperInterval":"PInt",
                   "Interval":"Int"}
    return '|'.join([abbrev_dict[term] for term in term_list])

In [14]:
abbrev(["Point", "ProperInterval"])

'Pt|PInt'

In [15]:
abbrev(['Point'])

'Pt'

In [16]:
class Algebra(object):

    def __init__(self, filename):
        """An algebra is created from a JSON file containing the algebra's
        relation and transitivity table definitions.
        """
        with open(filename, 'r') as f:
            self.algebra_dict = json.load(f)

        self.name = self.algebra_dict["Name"]
        self.description = self.algebra_dict["Description"]
        # TODO: For consistency, rename rel_info to rel_dict
        self.rel_info = self.algebra_dict["Relations"]
        # TODO: Maybe rename AlgBitSet to, say, alg_rels (or allrels_bitset, or ...)
        self.AlgBitSet = bitset(self.name, tuple(self.rel_info.keys()))  # A class object
        # TODO: Remove identity, and just use AlgBitSet.supremum
        self.identity = self.AlgBitSet.supremum

        # Setup the transitivity table used by Relation Set multiplication
        self.transitivity_table = dict()
        tabledefs = self.algebra_dict["TransTable"]
        for rel1 in tabledefs:
            self.transitivity_table[rel1] = dict()
            for rel2 in tabledefs[rel1]:
                self.transitivity_table[rel1][rel2] = self.AlgBitSet(tuple(tabledefs[rel1][rel2]))
                
    # Accessors for information about a given relation, rel.
    def rel_name(self, rel):
        return self.rel_info[rel]["Name"]
    def rel_domain(self,rel):
        return self.rel_info[rel]["Domain"]
    def rel_range(self,rel):
        return self.rel_info[rel]["Range"]
    def rel_reflexive(self,rel):
        return self.rel_info[rel]["Reflexive"]
    def rel_symmetric(self,rel):
        return self.rel_info[rel]["Symmetric"]
    def rel_transitive(self, rel):
        return self.rel_info[rel]["Transitive"]
    
    def converse(self, rel_or_relset):
        '''Return the converse of a relation (str) or relation set (bitset).'''
        if isinstance(rel_or_relset, str):
            return self.rel_info[rel_or_relset]["Converse"]
        else:
            return self.AlgBitSet((self.converse(r) for r in rel_or_relset.members()))
        
    def __str__(self):
        """Return a string representation of the Algebra."""
        return f"<{self.name}: {self.description}>"
    
    def relset(self, elements):
        """Return a relation set (bitset) for the given elements."""
        return self.AlgBitSet(elements)
    
    def add(self, relset1, relset2):
        '''Addition for relation sets is equivalent to set intersection.'''
        return relset1.intersection(relset2)
    
    def mult(self, relset1, relset2):
        '''Multiplication is done, element-by-element, on the cross-product
        of the two sets using the algebra's transitivity table, and
        then reducing those results to a single relation set using set
        union.
        '''
        result = self.AlgBitSet.infimum  # the empty relation set
        for r1 in relset1:
            for r2 in relset2:
                result = result.union(self.transitivity_table[r1][r2])
        return result
    
    def check_multiplication_identity(self, verbose=False):
        """Check the validity of the multiplicative identity for every
        combination of singleton relset.  :param verbose: Print out
        the details of each test :return: True or False

        """
        count = 0
        result = True
        rels = self.identity
        for r in rels:
            r_rs = self.relset((r,))
            for s in rels:
                count += 1
                s_rs = self.relset((s,))
                prod1 = self.mult(r_rs, s_rs)
                prod2 = self.converse(self.mult(self.converse(s_rs), self.converse(r_rs)))
                if prod1 != prod2:
                    if verbose:
                        print("FAIL:")
                        print(f"      r    = {r_rs}")
                        print(f"      s    = {s_rs}")
                        print(f"( r *  s)  = {prod1}")
                        print(f"(si * ri)i = {prod2}")
                        print(f"{prod1} != {prod2}")
                    result = False
        if verbose:
            print(f"\n{self.name} -- Multiplication Identity Check:")
        if result:
            if verbose:
                print(f"PASSED . {count} products tested.")
        else:
            if verbose:
                print("FAILED. See FAILURE output above.")
        return result

    def print_info(self):
        print(f"  Algebra Name: {self.name}")
        print(f"   Description: {self.description}")
        # print(f" Equality Rels: {self.equality_relations}")
        print("     Relations:")
        print("{:>25s} {:>25s} {:>10s} {:>10s} {:>10s} {:>8s} {:>12s}".format("NAME (ABBREV)", "CONVERSE (ABBREV)",
                                                                              "REFLEXIVE", "SYMMETRIC", "TRANSITIVE",
                                                                              "DOMAIN", "RANGE"))
        # TODO: Vary spacing between columns based on max word lengths
        for r in self.identity:
            print(f"{self.rel_name(r):>19s} ({r:>3s}) " \
                  f"{self.rel_name(self.converse(r)):>19s} ({self.converse(r):>3s}) " \
                  f"{self.rel_reflexive(r)!s:>8} {self.rel_symmetric(r)!s:>10} {self.rel_transitive(r)!s:>10}" \
                  f"{abbrev(self.rel_domain(r))!s:>11} {abbrev(self.rel_range(r))!s:>13}")
        # TODO: Don't hardcode the legend below; make it depend on an abbreviations file (JSON)
        print("\nDomain & Range Abbreviations:")
        print("   Pt = Point")
        print(" PInt = Proper Interval")
        
    def is_associative(self, verbose=False):
        result = True
        countskipped = 0
        countok = 0
        countfailed = 0
        counttotal = 0
        rels = self.identity
        for _a in rels:
            for _b in rels:
                for _c in rels:
                    if verbose:
                        print(f"{_a} x {_b} x {_c} :")
                    if (set(self.rel_range(_a)) & set(self.rel_domain(_b))) & (set(self.rel_range(_b)) & set(self.rel_domain(_c))):
                        a_rs = self.relset((_a,))
                        b_rs = self.relset((_b,))
                        c_rs = self.relset((_c,))
                        prod_ab = a_rs * b_rs
                        prod_bc = b_rs * c_rs
                        prod_ab_c = prod_ab * c_rs
                        prod_a_bc = a_rs * prod_bc
                        if not (prod_ab_c == prod_a_bc):
                            if verbose:
                                print(f"  Associativity fails for a = {a_rs}, b = {b_rs}, c = {c_rs}")
                                print(f"    associativity check: {self.rel_range(_a)}::{self.rel_domain(_b)} {self.rel_range(_b)}::{self.rel_domain(_c)}")
                                print(f"    (a * b) * c = {prod_ab_c}")
                                print(f"    a * (b * c) = {prod_a_bc}")
                            countfailed += 1
                            counttotal += 1
                            result = False
                        else:
                            if verbose:
                                print("  Associativity OK")
                            countok += 1
                            counttotal += 1
                    else:
                        if verbose:
                            print(f"  Skipping associativity check: {self.rel_range(_a)}::{self.rel_domain(_b)} {self.rel_range(_b)}::{self.rel_domain(_c)}")
                        countskipped += 1
                        counttotal += 1
        print(f"\nTEST SUMMARY: {countok} OK, {countskipped} Skipped, {countfailed} Failed ({counttotal} Total)")
        numrels = len(rels)
        totaltests = numrels * numrels * numrels
        if (counttotal != totaltests):
            print(f"Test counts do not add up; Total should be {totaltests}")
        return result
    
    def is_associative_NEW(self, verbose=False):
        result = True
        countskipped = 0
        countok = 0
        countfailed = 0
        counttotal = 0
        rels = self.identity
        for _a in rels:
            for _b in rels:
                for _c in rels:
                    if verbose:
                        print(f"{_a} x {_b} x {_c} :")
                    a_rs = self.relset((_a,))
                    b_rs = self.relset((_b,))
                    c_rs = self.relset((_c,))
                    if (set(self.rel_range(_a)) & set(self.rel_domain(_b))):
                        prod_ab = a_rs * b_rs
                        prod_ab_c = prod_ab * c_rs
                        if (set(self.rel_range(_b)) & set(self.rel_domain(_c))):
                            prod_bc = b_rs * c_rs
                            prod_a_bc = a_rs * prod_bc
                            if not (prod_ab_c == prod_a_bc):
                                if verbose:
                                    print(f"  Associativity fails for a = {a_rs}, b = {b_rs}, c = {c_rs}")
                                    print(f"    associativity check: {self.rel_range(_a)}::{self.rel_domain(_b)} {self.rel_range(_b)}::{self.rel_domain(_c)}")
                                    print(f"    (a * b) * c = {prod_ab_c}")
                                    print(f"    a * (b * c) = {prod_a_bc}")
                                countfailed += 1
                                counttotal += 1
                                result = False
                            else:
                                if verbose:
                                    print("  Associativity OK")
                                countok += 1
                                counttotal += 1
                        else:
                            if verbose:
                                print(f"  Skipping associativity due to b x c: {self.rel_range(_b)}::{self.rel_domain(_c)}")
                            countskipped += 1
                            counttotal += 1
                    else:
                        if verbose:
                            print(f"  Skipping associativity due to a x b: {self.rel_range(_a)}::{self.rel_domain(_b)}")
                        countskipped += 1
                        counttotal += 1
        print(f"\nTEST SUMMARY: {countok} OK, {countskipped} Skipped, {countfailed} Failed ({counttotal} Total)")
        numrels = len(rels)
        totaltests = numrels * numrels * numrels
        if (counttotal != totaltests):
            print(f"Test counts do not add up; Total should be {totaltests}")
        return result

In [17]:
alg = Algebra(os.path.join(path, "Algebras/IntervalAlgebra.json"))  # Allen's algebra of proper time intervals
#alg = Algebra(os.path.join(path, "Algebras/LeftBranchingIntervalAndPointAlgebra.json"))
#alg = Algebra(os.path.join(path, "Algebras/RightBranchingIntervalAndPointAlgebra.json"))
#alg = Algebra(os.path.join(path, "Algebras/rcc8Algebra.json"))

print(alg)

<LinearTimeIntervalAlgebra: Allen's algebra of proper time intervals>


In [18]:
alg.converse(alg.relset(('B','M','OI')))

LinearTimeIntervalAlgebra(['BI', 'MI', 'O'])

In [19]:
alg.converse('B')

'BI'

In [20]:
alg.identity

LinearTimeIntervalAlgebra(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'S', 'SI'])

In [21]:
len(alg.identity)

13

In [22]:
alg.rel_name('B')

'Before'

In [23]:
#alg.transitivity_table

In [24]:
before = alg.relset(['B'])
during = alg.relset(['D'])

bxd = alg.mult(before, during)
bxd

LinearTimeIntervalAlgebra(['B', 'D', 'M', 'O', 'S'])

In [25]:
foobar = alg.relset(['D','M','F','SI'])
print(foobar.members())
alg.add(bxd,foobar)

('D', 'F', 'M', 'SI')


LinearTimeIntervalAlgebra(['D', 'M'])

In [26]:
str(foobar.members())

"('D', 'F', 'M', 'SI')"

In [27]:
print(before.complement().members())
print(bxd.complement().members())

('BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'S', 'SI')
('BI', 'DI', 'E', 'F', 'FI', 'MI', 'OI', 'SI')


In [28]:
alg.check_multiplication_identity()

True

See [Format String Syntax](https://docs.python.org/3/library/string.html#format-string-syntax)

In [29]:
alg.print_info()

  Algebra Name: LinearTimeIntervalAlgebra
   Description: Allen's algebra of proper time intervals
     Relations:
            NAME (ABBREV)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
             Before (  B)               After ( BI)    False      False       True       PInt          PInt
              After ( BI)              Before (  B)    False      False       True       PInt          PInt
             During (  D)            Contains ( DI)    False      False       True       PInt          PInt
           Contains ( DI)              During (  D)    False      False       True       PInt          PInt
             Equals (  E)              Equals (  E)     True       True       True       PInt          PInt
           Finishes (  F)         Finished-by ( FI)    False      False       True       PInt          PInt
        Finished-by ( FI)            Finishes (  F)    False      False       True       PInt          PInt
              Meets ( 

In [30]:
alg.is_associative()


TEST SUMMARY: 2197 OK, 0 Skipped, 0 Failed (2197 Total)


True

In [31]:
alg.is_associative_NEW()


TEST SUMMARY: 2197 OK, 0 Skipped, 0 Failed (2197 Total)


True

## Networks

### Old Network Definition

In [34]:
class Network_OLD(object):
    """A network of entities (e.g., events) with relationships between the entities.
    The relationships must be defined by a single, specific algebra.
    """

    def __init__(self, algebra, network_name=None):
        self.__algebra = algebra
        self.__constraints = dict()
        self.__entities = set([])
        if network_name:
            self.__name = network_name
        else:
            self.__name = uuid.uuid4()

    def __len__(self):
        return len(self.__entities)

    @property
    def name(self):
        return str(self.__name)

    def __repr__(self):
        return f"<Network: {self.name}, {self.__entities} entities>"

    def __add_constraint(self, e1, e2, rs):
        """
        Add a constraint, rs (relset), between two entities (e1 & e2).
        """
        if e1 not in self.__constraints:
            self.__constraints[e1] = {e2: rs}
        else:
            if e2 not in self.__constraints[e1]:
                self.__constraints[e1][e2] = rs
            else:
                self.__constraints[e1][e2].union(rs)

    def __set_unconstrained_values(self):
        ident = self.__algebra.identity
        for ent1 in self.__entities:
            for ent2 in self.__entities:
                if ent1 in self.__constraints:
                    if ent2 in self.__constraints[ent1]:
                        pass
                    else:
                        self.constraint(ent1, ent2, ident)
                else:
                    if ent2 in self.__constraints:
                        if ent1 in self.__constraints[ent2]:
                            pass
                        else:
                            self.constraint(ent1, ent2, ident)
                    else:
                        self.constraint(ent1, ent2, ident)

    def constraint(self, entity1, entity2, rels=None):
        """Assert that entity1 relates to entity2 in one of the ways
        contained in the set of relations, rels.  For example: (e1
        [before, overlaps, meets] e2) iff (e1 before e2) OR (e1
        overlaps e2) OR (e1 meets e2).
        """

        # Remember Entities:
        self.__entities.add(entity1)
        self.__entities.add(entity2)

        if rels:
            relset = self.__algebra.relset(rels)
        else:
            relset = self.__algebra.identity

        self.__add_constraint(entity1, entity2, relset)
        self.__add_constraint(entity2, entity1, relset.converse)

        equality1 = [self.__algebra.equality_relation_dict[t] for t in entity1.gettype()]
        equality2 = [self.__algebra.equality_relation_dict[t] for t in entity2.gettype()]

        self.__add_constraint(entity1, entity1, RelationSet(equality1, self.__algebra))
        self.__add_constraint(entity2, entity2, RelationSet(equality2, self.__algebra))

    def propagate(self, verbose=False):
        """Propagate constraints in the network.
        @param verbose: Print number of loops as constraints are propagated.
        """
        loop_count = 0
        self.__set_unconstrained_values()
        something_changed = True  # Start off with this True so we'll loop at least once
        while something_changed:
            something_changed = False  # Immediately set to False; if nothing changes, we'll only loop once
            loop_count += 1
            for ent1 in self.__entities:
                for ent2 in self.__entities:
                    prod = self.__algebra.identity
                    c12 = self.__constraints[ent1][ent2]
                    for ent3 in self.__entities:
                        c13 = self.__constraints[ent1][ent3]
                        c32 = self.__constraints[ent3][ent2]
                        prod = prod + (c13 * c32)
                    if prod != c12:
                        something_changed = True  # Need to continue top-level propagation loop
                    self.__constraints[ent1][ent2] = prod
        if verbose:
            print(f"Number of propagation loops: {loop_count}")

    def print_constraints(self):
        print(f"\n{self}\nConstraints: (Source, Target, RelationSet)")
        for x in self.__constraints:
            for y in self.__constraints[x]:
                rels = self.__constraints[x][y]
                print(f"  {x.name}, {y.name}, {sorted(list(rels))}")

In [35]:
class TemporalObject(object):

    def __init__(self, types, name=None, start=None, end=None, dur=None):
        self.__type = types
        self.__name = name
        self.__start = start
        self.__end = end
        self.__duration = dur

    @property
    def name(self):
        return self.__name

    def gettype(self):
        return self.__type

    def settype(self, typ):
        self.__type = typ

    def __repr__(self):
        if self.__name:
            return f"<TemporalObject {self.__name} {self.__type}>"
        else:
            return f"<TemporalObject {self.__type}>"

### New Network Definition

In [33]:
class Network(nx.Graph):
    pass

In [36]:
evt1 = TemporalObject(["ProperInterval"], name="Event1")
evt2 = TemporalObject(["ProperInterval"], name="Event2")
print(evt1)
print(evt2)

<TemporalObject Event1 ['ProperInterval']>
<TemporalObject Event2 ['ProperInterval']>


In [37]:
net0 = Network()

In [38]:
net0.add_edge(evt1, evt2, object=foobar)

In [39]:
list(net0.edges)

[(<TemporalObject Event1 ['ProperInterval']>,
  <TemporalObject Event2 ['ProperInterval']>)]

In [40]:
list(net0.nodes)

[<TemporalObject Event1 ['ProperInterval']>,
 <TemporalObject Event2 ['ProperInterval']>]

In [41]:
class SpatialObject(object):

    def __init__(self, types, name=None):
        self.__type = types
        self.__name = name

    @property
    def name(self):
        return self.__name

    def gettype(self):
        return self.__type

    def settype(self, typ):
        self.__type = typ

    def __repr__(self):
        if self.__name:
            return f"<SpatialObject {self.__name} {self.__type}>"
        else:
            return f"<SpatialObject {self.__type}>"

In [42]:
#print("{}:".format(alg[0].name))
entity_x = TemporalObject(["ProperInterval"], "X")
entity_y = TemporalObject(["ProperInterval"], "Y")
entity_z = TemporalObject(["ProperInterval"], "Z")
r12 = alg.relset(["B"])
r23 = alg.relset(["D"])
print("\n")
print("Constraint: {} {} {}".format(entity_x.name, list(r12.members()), entity_y.name))
print("Constraint: {} {} {}".format(entity_y.name, list(r23.members()), entity_z.name))
net0 = Network(alg, "Test0")
net0.constraint(entity_x, entity_y, [r12])
net0.constraint(entity_y, entity_z, [r23])
net0.print_constraints()
net0.propagate(verbose=True)
net0.print_constraints()
print("\n")



Constraint: X ['B'] Y
Constraint: Y ['D'] Z


TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given